# Comparação de Algoritmos de Busca em Grafos — IA UFOP

**Problema:** Roteamento em Ouro Preto com função de custo (declividade, rugosidade, congestionamento).

**Algoritmos:** Dijkstra, A* e D* Lite.

**Cenários:** baseline, evento (ruas interditadas), clima (chuva — ladeiras com peso 200%).

In [4]:
import sys, os
sys.path.insert(0, '.')

from src.build_graph import BuildGraph
from src.test_cases import TestCases
from src.algorithms import dijkstra, a_star, d_star_lite
from src.metrics import measure_latency_ms
from src.helpers.graph_helper import display_graph

builder = BuildGraph()

## 1. Grafo de exemplo (Ouro Preto)

Vértices: Praça Tiradentes, Terminal, Campus UFOP, Rua São José, Diogo de Vasconcelos, Rua do Pilar, Xavier da Veiga, Ladeira da Barra.
Arestas: distância, declividade (%), rugosidade (pé de moleque ~1.57), volume/capacidade.

In [5]:
api_key = os.getenv('GOOGLE_MAPS_API_KEY')
G = builder.build_op_graph(levels=300, use_cache=True)

START, GOAL = 'rua_joão_pedro_da_silva', 'rua_rio_piracicaba'

print('Vértices:', list(G.nodes()))
print('Arestas (origem -> destino):', list(G.edges()))

build_op_graph: carregado do cache (1629 ruas, 5172 arestas)
Vértices: ['rua_rio_piracicaba', 'avenida_das_andorinhas', 'estrada_do_mosteiro', 'way_196416349', 'way_385991379', 'rua_rio_verde', 'way_385994083', 'way_385994085', 'way_386020340', 'praça_frei_luiz_maria_sartori', 'rua_vinte_e_quatro_de_julho', 'trilha_bau_cachoeira_das_andorinhas', 'avenida_das_andorinhas_386207748', 'way_115208211', 'way_836343202', 'way_115208253', 'way_196416342', 'rua_rio_acima', 'rua_rio_negro', 'rua_rio_doce', 'rua_rio_das_velhas', 'way_196611405', 'way_196611393', 'way_385994082', 'way_115208080', 'rua_santo_antônio', 'travessa_são_bento', 'way_115208155', 'way_115208180', 'rua_são_pedro', 'rua_são_gonçalo', 'way_115208239', 'rua_serra_negra', 'rua_sé_da_bahia', 'rua_vitorino_gonçalves_reis', 'way_196416344', 'way_196416355', 'rua_josé_eugênio_cabral', 'way_386203146', 'way_381559097', 'way_196416331', 'way_378678206', 'way_690941411', 'way_722103376', 'way_722103377', 'way_1091600383', 'way_386207

In [ ]:

# Gera grafo interativo (Dash Cytoscape) e exibe no notebook
display_graph(G, html_path="grafo_ouro_preto.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

## 2. Cenário baseline — comparação dos três algoritmos

In [6]:
TestCases.reset_scenarios(G)

lat_d, (path_d, cost_d) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), 200)
lat_a, (path_a, cost_a) = measure_latency_ms(lambda: a_star(G, START, GOAL), 200)
lat_dl, (path_dl, cost_dl) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), 200)
display_graph(G, html_path="grafo_ouro_preto.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500, start=START, goal=GOAL)

print('Dijkstra:  caminho =', path_d, '| custo =', round(cost_d, 2))
print('A*:       caminho =', path_a, '| custo =', round(cost_a, 2))
print('D* Lite:  caminho =', path_dl, '| custo =', round(cost_dl, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d:.4f} ms | A*: {lat_a:.4f} ms | D* Lite: {lat_dl:.4f} ms')

Dijkstra:  caminho = ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua_antônio_martins_32827034', 'rua_antônio_martins_194627902', 'rua_antônio_martins', 'rua_felipe_dos_santos', 'rua_carlos_tomaz', 'rua_cláudio_manoel', 'rua_antônio_pereira', 'rua_barão_de_camargos', 'rua_professor_ros_p_gomes', 'travessa_roque_de_paiva', 'rua_henrique_goerceix', 'ladeira_joão_de_paiva_194479756', 'way_385989649', 'ladeira_joão_de_paiva', 'rua_rio_negro', 'way_385994082', 'rua_rio_verde', 'rua_rio_piracicaba'] | custo = 4445.82
A*:       caminho = ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua_antônio_martins_32827034', 'rua_antônio_martins_194627902', 'rua_antônio_mar

## 3. Casos de teste com impeditivos no caminho (Ouro Preto)

Mantendo **START=rua_rio_piracicaba** e **GOAL=avenida_américo_rené_gianetti** (nós do grafo grafo_op_osm). Cada caso aplica interdições, congestionamento ou chuva em ruas do caminho baseline (rua_rio_piracicaba → ladeira_joão_de_paiva → rua_hugo_soderi → avenida_américo_rené_gianetti) para forçar desvio ou custo maior.

In [5]:
def run_case(name, setup_fn):
    """Aplica cenário, roda os 3 algoritmos e retorna (path_d, cost_d, path_a, cost_a, path_dl, cost_dl)."""
    TestCases.reset_scenarios(G)
    setup_fn(G)
    l_d, (path_d, cost_d) = measure_latency_ms(lambda: dijkstra(G, START, GOAL))
    l_a, (path_a, cost_a) = measure_latency_ms(lambda: a_star(G, START, GOAL))
    l_dl, (path_dl, cost_dl) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL))
    print(f"  {name}")
    print(f"    Dijkstra:  {path_d} | latencia = {l_d:.4f} ms | custo = {round(cost_d, 2)}")
    print(f"    A*:        {path_a} | latencia = {l_a:.4f} ms | custo = {round(cost_a, 2)}")
    print(f"    D* Lite:   {path_dl} | latencia = {l_dl:.4f} ms | custo = {round(cost_dl, 2)}")
    return path_d, cost_d, path_a, cost_a, path_dl, cost_dl

In [6]:
# Caso 1: interdição da primeira aresta do caminho (rua_rio_piracicaba → rua_rio_verde)
path_d, cost_d, path_a, cost_a, path_dl, cost_dl = run_case(
    "Caso 1 — interdição (primeira aresta do caminho)",
    lambda g: TestCases.apply_barriers(g, TestCases.OURO_PRETO_BLOCKED_CASE1),
)
display_graph(G, path=path_d, start=START, goal=GOAL, html_path="grafo_caso1.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

  Caso 1 — interdição (primeira aresta do caminho)
    Dijkstra:  ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua_antônio_martins_32827034', 'rua_antônio_martins_194627902', 'rua_antônio_martins', 'rua_felipe_dos_santos', 'rua_carlos_tomaz', 'rua_cláudio_manoel', 'rua_antônio_pereira', 'rua_barão_de_camargos', 'rua_professor_ros_p_gomes', 'travessa_roque_de_paiva', 'rua_henrique_goerceix', 'ladeira_joão_de_paiva_194479756', 'way_385989649', 'ladeira_joão_de_paiva', 'rua_rio_negro', 'way_385994082', 'rua_rio_verde', 'rua_rio_piracicaba'] | latencia = 10.9519 ms | custo = 4445.82
    A*:        ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua_antônio_mart

In [7]:
# Caso 2: congestionamento em trechos do caminho
path_d, cost_d, path_a, cost_a, path_dl, cost_dl = run_case(
    "Caso 2 — congestionamento (ladeira_joão_de_paiva↔rua_hugo_soderi, rua_hugo_soderi↔avenida_américo_rené_gianetti)",
    lambda g: TestCases.apply_congestion_scenario_by_edge(g, TestCases.OURO_PRETO_CONGESTED_CASE2),
)
display_graph(G, path=path_d, start=START, goal=GOAL, html_path="grafo_caso2.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

  Caso 2 — congestionamento (ladeira_joão_de_paiva↔rua_hugo_soderi, rua_hugo_soderi↔avenida_américo_rené_gianetti)
    Dijkstra:  ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua_antônio_martins_32827034', 'rua_antônio_martins_194627902', 'rua_antônio_martins', 'rua_felipe_dos_santos', 'rua_carlos_tomaz', 'rua_cláudio_manoel', 'rua_antônio_pereira', 'rua_barão_de_camargos', 'rua_professor_ros_p_gomes', 'travessa_roque_de_paiva', 'rua_henrique_goerceix', 'ladeira_joão_de_paiva_194479756', 'way_385989649', 'ladeira_joão_de_paiva', 'rua_rio_negro', 'way_385994082', 'rua_rio_verde', 'rua_rio_piracicaba'] | latencia = 7.9140 ms | custo = 4445.82
    A*:        ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way

In [8]:
# Caso 3: chuva em regiões (ladeira_joão_de_paiva, rua_hugo_soderi, avenida_américo_rené_gianetti)
path_d, cost_d, path_a, cost_a, path_dl, cost_dl = run_case(
    "Caso 3 — chuva em regiões do caminho",
    lambda g: TestCases.apply_climate_scenario_by_region(g, TestCases.OURO_PRETO_RAIN_REGIONS_CASE3),
)
display_graph(G, path=path_d, start=START, goal=GOAL, html_path="grafo_caso3.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

  Caso 3 — chuva em regiões do caminho
    Dijkstra:  ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua_antônio_martins_32827034', 'rua_antônio_martins_194627902', 'rua_antônio_martins', 'rua_felipe_dos_santos', 'rua_carlos_tomaz', 'rua_cláudio_manoel', 'rua_antônio_pereira', 'rua_barão_de_camargos', 'rua_professor_ros_p_gomes', 'travessa_roque_de_paiva', 'rua_henrique_goerceix', 'ladeira_joão_de_paiva_194479756', 'way_385989649', 'ladeira_joão_de_paiva', 'rua_rio_negro', 'way_385994082', 'rua_rio_verde', 'rua_rio_piracicaba'] | latencia = 14.1090 ms | custo = 4445.82
    A*:        ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua_antônio_martins_32827034

In [9]:
# Caso 4: interdição + congestionamento
def setup_case4(g):
    TestCases.apply_barriers(g, TestCases.OURO_PRETO_BLOCKED_CASE4)
    TestCases.apply_congestion_scenario_by_edge(g, TestCases.OURO_PRETO_CONGESTED_CASE4)
path_d, cost_d, path_a, cost_a, path_dl, cost_dl = run_case("Caso 4 — interdição + congestionamento", setup_case4)
display_graph(G, path=path_d, start=START, goal=GOAL, html_path="grafo_caso4.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

  Caso 4 — interdição + congestionamento
    Dijkstra:  ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua_antônio_martins_32827034', 'rua_antônio_martins_194627902', 'rua_antônio_martins', 'rua_felipe_dos_santos', 'rua_carlos_tomaz', 'rua_cláudio_manoel', 'rua_antônio_pereira', 'rua_barão_de_camargos', 'rua_professor_ros_p_gomes', 'travessa_roque_de_paiva', 'rua_henrique_goerceix', 'ladeira_joão_de_paiva_194479756', 'way_385989649', 'ladeira_joão_de_paiva', 'rua_rio_negro', 'way_385994082', 'rua_rio_verde', 'rua_rio_piracicaba'] | latencia = 8.1964 ms | custo = 4445.82
    A*:        ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua_antônio_martins_3282703

In [10]:
# Caso 5: interdição ladeira_joão_de_paiva ↔ rua_hugo_soderi
path_d, cost_d, path_a, cost_a, path_dl, cost_dl = run_case(
    "Caso 5 — interdição (ladeira_joão_de_paiva ↔ rua_hugo_soderi)",
    lambda g: TestCases.apply_barriers(g, TestCases.OURO_PRETO_BLOCKED_CASE5),
)
display_graph(G, path=path_d, start=START, goal=GOAL, html_path="grafo_caso5.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

  Caso 5 — interdição (ladeira_joão_de_paiva ↔ rua_hugo_soderi)
    Dijkstra:  ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua_antônio_martins_32827034', 'rua_antônio_martins_194627902', 'rua_antônio_martins', 'rua_felipe_dos_santos', 'rua_carlos_tomaz', 'rua_cláudio_manoel', 'rua_antônio_pereira', 'rua_barão_de_camargos', 'rua_professor_ros_p_gomes', 'travessa_roque_de_paiva', 'rua_henrique_goerceix', 'ladeira_joão_de_paiva_194479756', 'way_385989649', 'ladeira_joão_de_paiva', 'rua_rio_negro', 'way_385994082', 'rua_rio_verde', 'rua_rio_piracicaba'] | latencia = 10.6353 ms | custo = 4445.82
    A*:        ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua

In [11]:
# Caso 6: congestionamento em todas as arestas do caminho baseline
path_d, cost_d, path_a, cost_a, path_dl, cost_dl = run_case(
    "Caso 6 — congestionamento em todo o caminho baseline",
    lambda g: TestCases.apply_congestion_scenario_by_edge(g, TestCases.OURO_PRETO_CONGESTED_CASE6),
)
display_graph(G, path=path_d, start=START, goal=GOAL, html_path="grafo_caso6.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

  Caso 6 — congestionamento em todo o caminho baseline
    Dijkstra:  ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua_antônio_martins_32827034', 'rua_antônio_martins_194627902', 'rua_antônio_martins', 'rua_felipe_dos_santos', 'rua_carlos_tomaz', 'rua_cláudio_manoel', 'rua_antônio_pereira', 'rua_barão_de_camargos', 'rua_professor_ros_p_gomes', 'travessa_roque_de_paiva', 'rua_henrique_goerceix', 'ladeira_joão_de_paiva_194479756', 'way_385989649', 'ladeira_joão_de_paiva', 'rua_rio_negro', 'way_385994082', 'rua_rio_verde', 'rua_rio_piracicaba'] | latencia = 9.5411 ms | custo = 4445.82
    A*:        ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua_antônio_m

In [12]:
# Caso 7: interdição rua_hugo_soderi↔avenida_américo_rené_gianetti + chuva em regiões
def setup_case7(g):
    TestCases.apply_barriers(g, TestCases.OURO_PRETO_BLOCKED_CASE7)
    TestCases.apply_climate_scenario_by_region(g, TestCases.OURO_PRETO_RAIN_REGIONS_CASE7)
path_d, cost_d, path_a, cost_a, path_dl, cost_dl = run_case("Caso 7 — interdição + chuva em regiões", setup_case7)
display_graph(G, path=path_d, start=START, goal=GOAL, html_path="grafo_caso7.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

  Caso 7 — interdição + chuva em regiões
    Dijkstra:  ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua_antônio_martins_32827034', 'rua_antônio_martins_194627902', 'rua_antônio_martins', 'rua_felipe_dos_santos', 'rua_carlos_tomaz', 'rua_cláudio_manoel', 'rua_antônio_pereira', 'rua_barão_de_camargos', 'rua_professor_ros_p_gomes', 'travessa_roque_de_paiva', 'rua_henrique_goerceix', 'ladeira_joão_de_paiva_194479756', 'way_385989649', 'ladeira_joão_de_paiva', 'rua_rio_negro', 'way_385994082', 'rua_rio_verde', 'rua_rio_piracicaba'] | latencia = 8.9648 ms | custo = 4445.82
    A*:        ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua_antônio_martins_3282703

## 3. Cenário de evento — ruas interditadas (Diogo de Vasconcelos)

O sistema deve sugerir rotas alternativas (Rua do Pilar ou Xavier da Veiga).

In [13]:
TestCases.apply_event_scenario(G)

lat_d_e, (path_d_e, cost_d_e) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), 200)
lat_a_e, (path_a_e, cost_a_e) = measure_latency_ms(lambda: a_star(G, START, GOAL), 200)
lat_dl_e, (path_dl_e, cost_dl_e) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), 200)

print('Com ruas interditadas:')
print('Dijkstra:  caminho =', path_d_e, '| custo =', round(cost_d_e, 2))
print('A*:       caminho =', path_a_e, '| custo =', round(cost_a_e, 2))
print('D* Lite:  caminho =', path_dl_e, '| custo =', round(cost_dl_e, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_e:.4f} ms | A*: {lat_a_e:.4f} ms | D* Lite: {lat_dl_e:.4f} ms')

Com ruas interditadas:
Dijkstra:  caminho = ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua_antônio_martins_32827034', 'rua_antônio_martins_194627902', 'rua_antônio_martins', 'rua_felipe_dos_santos', 'rua_carlos_tomaz', 'rua_cláudio_manoel', 'rua_antônio_pereira', 'rua_barão_de_camargos', 'rua_professor_ros_p_gomes', 'travessa_roque_de_paiva', 'rua_henrique_goerceix', 'ladeira_joão_de_paiva_194479756', 'way_385989649', 'ladeira_joão_de_paiva', 'rua_rio_negro', 'way_385994082', 'rua_rio_verde', 'rua_rio_piracicaba'] | custo = 4445.82
A*:       caminho = ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua_antônio_martins_32827034', 'rua_antônio_martins_19462

## 4. Cenário climático — chuva (ladeiras íngremes +200%)

In [14]:
TestCases.reset_scenarios(G)
TestCases.apply_climate_scenario(G, rain_multiplier=2.0)

lat_d_c, (path_d_c, cost_d_c) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), 200)
lat_a_c, (path_a_c, cost_a_c) = measure_latency_ms(lambda: a_star(G, START, GOAL), 200)
lat_dl_c, (path_dl_c, cost_dl_c) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), 200)

print('Com chuva (ladeiras +200%):')
print('Dijkstra:  caminho =', path_d_c, '| custo =', round(cost_d_c, 2))
print('A*:       caminho =', path_a_c, '| custo =', round(cost_a_c, 2))
print('D* Lite:  caminho =', path_dl_c, '| custo =', round(cost_dl_c, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_c:.4f} ms | A*: {lat_a_c:.4f} ms | D* Lite: {lat_dl_c:.4f} ms')

Com chuva (ladeiras +200%):
Dijkstra:  caminho = ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua_antônio_martins_32827034', 'rua_antônio_martins_194627902', 'rua_antônio_martins', 'rua_felipe_dos_santos', 'rua_carlos_tomaz', 'rua_cláudio_manoel', 'rua_antônio_pereira', 'rua_barão_de_camargos', 'rua_professor_ros_p_gomes', 'travessa_roque_de_paiva', 'rua_henrique_goerceix', 'ladeira_joão_de_paiva_194479756', 'way_385989649', 'ladeira_joão_de_paiva', 'rua_rio_negro', 'way_385994082', 'rua_rio_verde', 'rua_rio_piracicaba'] | custo = 4445.82
A*:       caminho = ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua_antônio_martins_32827034', 'rua_antônio_martins_

### 4.1 Cenários combinados (Ouro Preto)

Bloqueio + chuva; bloqueio + chuva + alta densidade; chuva e congestionamento **por região** (alguns bairros com chuva/trânsito, outros não).

In [15]:
# Cenário A: bloqueio (Diogo de Vasconcelos) + chuva
TestCases.reset_scenarios(G)
TestCases.apply_event_scenario(G)
TestCases.apply_climate_scenario(G, rain_multiplier=2.0)

lat_d_ac, (path_d_ac, cost_d_ac) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), 200)
lat_a_ac, (path_a_ac, cost_a_ac) = measure_latency_ms(lambda: a_star(G, START, GOAL), 200)
lat_dl_ac, (path_dl_ac, cost_dl_ac) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), 200)
display_graph(G, start=START, goal=GOAL, path=path_d_ac, html_path="grafo_ouro_preto.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

print('A — Bloqueio + Chuva:')
print('  Dijkstra:', path_d_ac, '| custo =', round(cost_d_ac, 2))
print('  A*:     ', path_a_ac, '| custo =', round(cost_a_ac, 2))
print('  D* Lite:', path_dl_ac, '| custo =', round(cost_dl_ac, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_ac:.4f} ms | A*: {lat_a_ac:.4f} ms | D* Lite: {lat_dl_ac:.4f} ms')

A — Bloqueio + Chuva:
  Dijkstra: ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua_antônio_martins_32827034', 'rua_antônio_martins_194627902', 'rua_antônio_martins', 'rua_felipe_dos_santos', 'rua_carlos_tomaz', 'rua_cláudio_manoel', 'rua_antônio_pereira', 'rua_barão_de_camargos', 'rua_professor_ros_p_gomes', 'travessa_roque_de_paiva', 'rua_henrique_goerceix', 'ladeira_joão_de_paiva_194479756', 'way_385989649', 'ladeira_joão_de_paiva', 'rua_rio_negro', 'way_385994082', 'rua_rio_verde', 'rua_rio_piracicaba'] | custo = 4445.82
  A*:      ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua_antônio_martins_32827034', 'rua_antônio_martins_194627902', 'rua_antônio

In [16]:
TestCases.reset_scenarios(G)
TestCases.apply_event_scenario(G)
TestCases.apply_climate_scenario(G, rain_multiplier=2.0)
TestCases.apply_congestion_scenario_by_edge(G, {("lagoa", "morro_santana"): 2.0})

lat_d_b, (path_d_b, cost_d_b) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), 200)
lat_a_b, (path_a_b, cost_a_b) = measure_latency_ms(lambda: a_star(G, START, GOAL), 200)
lat_dl_b, (path_dl_b, cost_dl_b) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), 200)
display_graph(G, path=path_d_b, start=START, goal=GOAL, html_path="grafo_ouro_preto.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)


print('B — Bloqueio + Chuva + Congestionamento:')
print('  Dijkstra:', path_d_b, '| custo =', round(cost_d_b, 2))
print('  A*:     ', path_a_b, '| custo =', round(cost_a_b, 2))
print('  D* Lite:', path_dl_b, '| custo =', round(cost_dl_b, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_b:.4f} ms | A*: {lat_a_b:.4f} ms | D* Lite: {lat_dl_b:.4f} ms')

B — Bloqueio + Chuva + Congestionamento:
  Dijkstra: ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua_antônio_martins_32827034', 'rua_antônio_martins_194627902', 'rua_antônio_martins', 'rua_felipe_dos_santos', 'rua_carlos_tomaz', 'rua_cláudio_manoel', 'rua_antônio_pereira', 'rua_barão_de_camargos', 'rua_professor_ros_p_gomes', 'travessa_roque_de_paiva', 'rua_henrique_goerceix', 'ladeira_joão_de_paiva_194479756', 'way_385989649', 'ladeira_joão_de_paiva', 'rua_rio_negro', 'way_385994082', 'rua_rio_verde', 'rua_rio_piracicaba'] | custo = 4445.82
  A*:      ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua_antônio_martins_32827034', 'rua_antônio_martins_19462

In [17]:
# Cenário C: chuva por região + congestionamento por via (vias centro↔são_jose congestionadas)
TestCases.reset_scenarios(G)
TestCases.apply_climate_scenario_by_region(G, {"centro":9.0, "praca_tiradentes": 9.0, "campus": 1.0})
TestCases.apply_congestion_scenario_by_edge(G, {("centro", "sao_jose"): 5.0, ("sao_jose", "centro"): 5.0})
display_graph(G, start=START, goal=GOAL, html_path="grafo_ouro_preto.html", display_in_notebook=True, iframe_height=760, iframe_width=960, height=500)

lat_d_c2, (path_d_c2, cost_d_c2) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), 200)
lat_a_c2, (path_a_c2, cost_a_c2) = measure_latency_ms(lambda: a_star(G, START, GOAL), 200)
lat_dl_c2, (path_dl_c2, cost_dl_c2) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), 200)

print('C — Chuva por região + Congestionamento por via (centro↔são_jose congestionados):')
print('  Dijkstra:', path_d_c2, '| custo =', round(cost_d_c2, 2))
print('  A*:     ', path_a_c2, '| custo =', round(cost_a_c2, 2))
print('  D* Lite:', path_dl_c2, '| custo =', round(cost_dl_c2, 2))
print(f'  Latência média (200 exec.): Dijkstra: {lat_d_c2:.4f} ms | A*: {lat_a_c2:.4f} ms | D* Lite: {lat_dl_c2:.4f} ms')

C — Chuva por região + Congestionamento por via (centro↔são_jose congestionados):
  Dijkstra: ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua_antônio_martins_32827034', 'rua_antônio_martins_194627902', 'rua_antônio_martins', 'rua_felipe_dos_santos', 'rua_carlos_tomaz', 'rua_cláudio_manoel', 'rua_antônio_pereira', 'rua_barão_de_camargos', 'rua_professor_ros_p_gomes', 'travessa_roque_de_paiva', 'rua_henrique_goerceix', 'ladeira_joão_de_paiva_194479756', 'way_385989649', 'ladeira_joão_de_paiva', 'rua_rio_negro', 'way_385994082', 'rua_rio_verde', 'rua_rio_piracicaba'] | custo = 4445.82
  A*:      ['rua_joão_pedro_da_silva', 'way_193208512', 'rua_professor_paulo_magalhães_gomes', 'rua_professor_geraldo_nunes', 'rua_pandiá_calógeras', 'rua_pandiá_calogeras', 'way_194569031', 'rua_antônio_martins_945090263', 'rua_antônio_mart

## 5. Latência de re-roteamento (< 100 ms para tempo real)

In [18]:
TestCases.reset_scenarios(G)
repetitions = 500

lat_d, (path_d, cost_d) = measure_latency_ms(lambda: dijkstra(G, START, GOAL), repetitions)
lat_a, (path_a, cost_a) = measure_latency_ms(lambda: a_star(G, START, GOAL), repetitions)
lat_dl, (path_dl, cost_dl) = measure_latency_ms(lambda: d_star_lite(G, START, GOAL), repetitions)

print(f'Latência média ({repetitions} execuções):')
print(f'  Dijkstra: {lat_d:.4f} ms')
print(f'  A*:       {lat_a:.4f} ms')
print(f'  D* Lite:  {lat_dl:.4f} ms')
print('  Meta: < 100 ms para uso em tempo real')

Latência média (500 execuções):
  Dijkstra: 9.5376 ms
  A*:       2.5331 ms
  D* Lite:  2.3181 ms
  Meta: < 100 ms para uso em tempo real


## 6. Tabela resumo para o relatório

In [19]:
import pandas as pd

TestCases.reset_scenarios(G)
rows = []
for name, fn in [('Dijkstra', dijkstra), ('A*', a_star), ('D* Lite', d_star_lite)]:
    lat, (path, cost) = measure_latency_ms(lambda: fn(G, START, GOAL), 200)
    rows.append({'Algoritmo': name, 'Custo': round(cost, 2), 'Latência (ms)': round(lat, 4)})

df = pd.DataFrame(rows)
df

,Algoritmo,Custo,Latência (ms)
0,Dijkstra,4445.82,7.2981
1,A*,4445.82,2.5310
2,D* Lite,4445.82,2.3001
